### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = today

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
6677,2023-11-23,Libano Divisão 1,14:00,Sagesse Al Hekmeh Beirut,Mayrouba,1.04,9.35,155.5,1.80,1.90,-16.5,1.81,1.89,b13fKw81,0.961538,0.106952,0.555556,0.526316,0.068490,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,89,73,2.31,1.56,0.000,0.000,1.131099,0.038222,0.030578,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
6678,2023-11-23,Lituânia Nkl,11:00,Telsiai,Silute,1.11,5.69,157.5,1.83,1.83,-13.5,2.00,1.72,2shTiwZS,0.900901,0.175747,0.546448,0.546448,0.076648,85.188,11.378320,0.133567,2.4,1.341641,0.559017,91.80,1.090,0.076485,0.070170,58.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,85,77,1.08,2.53,0.000,0.000,0.952514,0.000000,0.106446,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
6679,2023-11-23,Islândia Premier League,16:15,Tindastoll,Haukar,1.42,2.72,176.5,1.86,1.84,-6.5,2.02,1.71,zgPzF7IP,0.704225,0.367647,0.537634,0.543478,0.071872,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,97,79,1.49,3.79,0.000,0.000,0.444077,0.007644,0.117535,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
6680,2023-11-23,Islândia Premier League,16:15,Njardvik,Thorl,1.59,2.26,181.5,1.84,1.86,-4.5,2.06,1.68,hUaj7eaJ,0.628931,0.442478,0.543478,0.537634,0.071409,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,101,120,2.48,1.03,0.000,0.000,0.246110,0.007644,0.143690,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
6681,2023-11-23,Islândia Premier League,16:15,Hottur,Stjarnan,2.97,1.36,168.5,1.87,1.79,5.5,1.97,1.74,d2QWFm3J,0.336700,0.735294,0.534759,0.558659,0.071994,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,69,89,4.89,1.23,0.000,0.000,0.525839,0.030912,0.087674,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6745,2023-11-23,Eua Ncaa,00:30,UTEP,Bradley,2.57,1.58,137.5,1.91,1.91,2.5,2.10,1.67,U9wnL2Ag,0.389105,0.632911,0.523560,0.523560,0.022016,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,75,80,1.71,1.83,0.000,0.000,0.337367,0.000000,0.161303,0.00,0.00,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
6746,2023-11-23,Coreia Do Sul Kbl,07:00,Seoul Thunders,KoGas,1.87,1.90,164.5,1.89,1.85,-1.5,1.98,1.74,2RFFKdNQ,0.534759,0.526316,0.529101,0.540541,0.061075,299.366,132.590327,0.442904,0.0,0.000000,NaN,286.50,4.142,1.766839,0.426567,-64.0,224.490,48.017906,0.213898,0.0,0.000000,NaN,265.05,2.888,0.803691,0.278287,-33.0,75,93,3.82,2.85,186.192,216.518,0.011254,0.015125,0.091240,-2.75,-0.55,-1.581818,0.0,0.0,0.0,-5.0,-1.0,-0.9,0.0,0.0,0.0
6747,2023-11-23,Coreia Do Sul Kbl,07:00,LG Sakers,Mobis Phoebus,1.44,2.76,155.5,

## Apostas do dia

### Back Home V1 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v1 = jogos.copy()

df_bh_v1 = df_bh_v1[(df_bh_v1['Odds_H'] >= 1.6) & (df_bh_v1['Odds_H'] <= 2.4)]

if not df_bh_v1.empty:    
    # Filtros pós modelo
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 0.0) | (df_bh_v1['CG_A_Last_Game'] >= 160))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CV_HA'] < 0.0334) | (df_bh_v1['CV_HA'] >= 1.414))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['P_Diff'] < 0.0464) | (df_bh_v1['P_Diff'] >= 0.065))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['CG_A_Last_Game'] < 348) | (df_bh_v1['CG_A_Last_Game'] >= 920))]
    df_bh_v1 = df_bh_v1[((df_bh_v1['Goals_Last_H'] >= 79) & (df_bh_v1['Goals_Last_H'] <= 110)) | ((df_bh_v1['Goals_Last_H'] > 120.8) & (df_bh_v1['Goals_Last_H'] <= 143))]

    if not df_bh_v1.empty:
        df_bh_v1.to_csv(f'apostas_do_dia/{data_var}/back_home_v1_{data_var}.csv', index=False)

        print('------------ MÉTODO BACK HOME V1 (QCUT) ------------')
        print()

        df_bh_v1['Bet'] = 'Back Home'
        ligs = df_bh_v1['League'].unique().tolist()

        display(df_bh_v1[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

Sem jogos
